In [ ]:
# OpenAI GYM
# Stable-baselines 3 
# Optuna
# AWS Sagemaker Stuido 


In [1]:
# Install system packages
!apt-get update && apt-get -y install unzip freeglut3-dev cmake libopenmpi-dev python3-dev zlib1g-dev swig python3-dev libopenmpi-dev xvfb x11-utils ffmpeg #remove -qq for full out

# Install pyton packages : OBS Box2D might require kernel restart
!pip3 install stable-baselines3[extra] box2d-kengz box2d-py box2d pyvirtualdisplay pyglet==1.5.0 numpy optuna 
# !pip3 uninstall box2d  -y

Hit:1 http://security.ubuntu.com/ubuntu xenial-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu xenial InRelease
Hit:9 http://archive.ubuntu.com/ubuntu xenial-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu xenial-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-dev is already the newest version (3.5.1-3).
unzip is already the newest version (6.0-20ubuntu1).
x11-utils is already the newest version (7.7+3).
freeglut3-dev is already the newest version (2.8.1-2).
libopenmpi-dev is alrea

In [1]:
import gym
import numpy as np
import base64
import IPython
import PIL.Image
import pyvirtualdisplay
import os
import optuna


# Video 
from pathlib import Path
from IPython import display as ipythondisplay

# Stable baselines

# DQN specific
from stable_baselines3 import DQN
from stable_baselines3.dqn import MlpPolicy
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

# Stable baselines 3
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common import results_plotter
from stable_baselines3.common.results_plotter import load_results, ts2xy, plot_results
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from stable_baselines3.common.vec_env import VecVideoRecorder, SubprocVecEnv, DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import CallbackList, BaseCallback, CheckpointCallback, EveryNTimesteps, EvalCallback, StopTrainingOnRewardThreshold, StopTrainingOnMaxEpisodes

In [2]:
# Enviorment 
env_id = 'LunarLander-v2'
env = gym.make(env_id)
eval_env = gym.make(env_id)

# Video 
video_folder = './videos'
video_length = 3000

# Logs
logs_base_dir = "./log" 
os.makedirs(logs_base_dir, exist_ok=True) 

# Vars
score = 0
steps_total= 0 
log_interval=100


print('State shape: ', env.observation_space.shape)
# print('Number of actions: ', env.action_space.n)


State shape:  (8,)


In [3]:
### Record & Display Video

import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

# Record video
def record_model(env_id, model, video_length=500, prefix='', video_folder='videos/'):
  """
  :param env_id: (str)
  :param model: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """
  eval_env = DummyVecEnv([lambda: gym.make(env_id)])
  eval_env = VecVideoRecorder(env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = eval_env.reset()
  for _ in range(video_length):
    action, _ = model.predict(obs)
    obs, _, _, _ = eval_env.step(action)
  eval_env.close()


## Display video
def display_videos(video_path='', prefix=''):
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

In [4]:
        # Training function
def train_model(name, steps=10000, save=False, prefix=env_id, eval=1000):
    model.learn(total_timesteps=steps, log_interval=log_interval)
    if save == True:
     model.save(name + "-" + prefix)

def record_video(name, length=1500):
    record_model(env_id, model, video_length=length, prefix=name)
    display_videos('videos', prefix=name)
    print(name, " steps total")

In [5]:
def run_training(steps_per_round=200000,limit=300, max_steps=999999999):

  
  global score
  global steps_total
  
  # reset steps_total & score before running score loop
  steps_total = 0
  score = 0
  
  
  while score < limit:
      steps_total = steps_total + steps_per_round
      train_model(str(steps_total), steps=steps_per_round)
      new_eval = evaluate_policy(model, env, n_eval_episodes=5, deterministic=True, render=False, callback=None, reward_threshold=None, return_episode_rewards=False)
      score = new_eval[0]
      print("Round completed, steps_total: ",steps_total, " mean: ", score )  
        
      # If score reached, break         
      if score >= limit:
          print("Score limit of ",score," reached")
          break
        
      # If steos limit reached, break
      if steps_total >= max_steps:
          print("Max steps limit of ",max_steps," with a total count of steps_total: ",steps_total,)
          break
    



  ep100 = evaluate_policy(model, env, n_eval_episodes=10, deterministic=True, render=False, callback=None, reward_threshold=None, return_episode_rewards=True)
  print("Mean reward 100 ep: ", ep100[0])
  print("Steps total:", steps_total)
  print(score)
    
  # Return total steps needed  
  return steps_total

In [8]:
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip

os.makedirs(logs_base_dir, exist_ok=True)
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(logs_base_dir))

get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://a550611c42de.ngrok.io


In [ ]:
### Optuna
def objective(trial):
    
    global model
    obs = env.reset()

    
    gamma = trial.suggest_categorical("gamma", [0.9, 0.95, 0.98, 0.99, 0.995, 0.999, 0.9999])
    learning_rate = trial.suggest_loguniform("lr", 0.000050, 0.000080)
#     batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 100, 128, 256, 512])
#     buffer_size = trial.suggest_categorical("buffer_size", [int(1e4), int(5e4), int(1e5), int(1e6)])
#     exploration_final_eps = trial.suggest_uniform("exploration_final_eps", 0, 0.2)
#     exploration_fraction = trial.suggest_uniform("exploration_fraction", 0, 0.5)
#     target_update_interval = trial.suggest_categorical("target_update_interval", [1, 1000, 5000, 10000, 15000, 20000])
#     learning_starts = trial.suggest_categorical("learning_starts", [1000, 5000, 10000])
#     train_freq = trial.suggest_categorical("train_freq", [1, 4, 8, 16, 128, 256, 1000])
#     subsample_steps = trial.suggest_categorical("subsample_steps", [1, 2, 4, 8])
#     gradient_steps = max(train_freq // subsample_steps, 1)
#     n_episodes_rollout = -1
#     net_arch = trial.suggest_categorical("net_arch", ["tiny", "small", "medium"])
#     net_arch = {"tiny": [64], "small": [64, 64], "medium": [256, 256]}[net_arch]
    
    

    model = DQN (
        MlpPolicy,
        env,
        gamma=gamma,
        learning_rate=learning_rate,
        batch_size=128, #batch_size,
        buffer_size=50000, #buffer_size,
        train_freq=4, #train_freq,
        gradient_steps=-1, #gradient_steps,
#         n_episodes_rollout=n_episodes_rollout,
        exploration_fraction=0.12, #exploration_fraction,
        exploration_final_eps=0.1, #exploration_final_eps,
#         target_update_interval=target_update_interval,
        learning_starts=0, #learning_starts,
        policy_kwargs=dict(net_arch=[256, 256])
#         tensorboard_log=logs_base_dir,
    )

 

    
    # Traing
    run_training(steps_per_round=25000,limit=200, max_steps=500000)
    del model
    return steps_total 



In [ ]:
env.reset()
# !apt-get install redis-server -y
!service redis-server start
# !conda install mysqldb
# !pip3 install redis
# !pip install redis 
# # import mysql.connector

# import redis


storage = optuna.storages.RedisStorage(
    url="redis://localhost:6379",
)
    
    
    
study = optuna.create_study(study_name="lunarlander-v2-1.2.0",storage=storage, load_if_exists=True)
study.optimize(objective, n_trials=5)
print(study.best_params)
print(study.best_trial)

Starting redis-server: redis-server.


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: ExperimentalWarning: RedisStorage is experimental (supported from v1.4.0). The interface can change in the future.
  del sys.path[0]
[I 2020-11-17 12:15:57,555] A new study created in Redis with name: lunarlander-v2-1.2.0


Round completed, steps_total:  25000  mean:  26.19634212669621
Round completed, steps_total:  50000  mean:  -16.203972237886326
Round completed, steps_total:  75000  mean:  -4.673993960920825
Round completed, steps_total:  100000  mean:  -50.829100618284066
Round completed, steps_total:  125000  mean:  -97.7190014895167
Round completed, steps_total:  150000  mean:  -113.46740149185109
Round completed, steps_total:  175000  mean:  -82.31469050632401
Round completed, steps_total:  200000  mean:  -80.99561880129382
Round completed, steps_total:  225000  mean:  -95.84695622458123
Round completed, steps_total:  250000  mean:  -87.13330502074179
Round completed, steps_total:  275000  mean:  -105.73155179695914
Round completed, steps_total:  300000  mean:  -123.06158705450203
Round completed, steps_total:  325000  mean:  -92.58344929622253
Round completed, steps_total:  350000  mean:  -126.58178198996465
Round completed, steps_total:  375000  mean:  -117.6844862431362
Round completed, steps_t